In [2]:
import json
from logging import Logger
import os
from typing import Dict, List

import numpy as np
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
import pandas as pd
from tensorboardX import SummaryWriter
import torch
from tqdm import trange
from torch.optim.lr_scheduler import ExponentialLR

# from .evaluate import evaluate, evaluate_predictions
# from .predict import predict
# from .train import train
# from .loss_functions import get_loss_func
from chemprop.spectra_utils import normalize_spectra, load_phase_mask
from chemprop.args import TrainArgs
from chemprop.constants import MODEL_FILE_NAME
from chemprop.data import get_class_sizes, get_data, MoleculeDataLoader, MoleculeDataset, set_cache_graph, split_data
from chemprop.models import MoleculeModel
from chemprop.nn_utils import param_count, param_count_all
from chemprop.utils import build_optimizer, build_lr_scheduler, load_checkpoint, makedirs, \
    save_checkpoint, save_smiles_splits, load_frzn_model, multitask_mean

In [50]:
from chemprop.models import MoleculeModel

In [5]:
bbbp_dir = '../../data/bbbp.csv'

In [6]:
data_process = get_data(bbbp_dir)

2039it [00:00, 244943.03it/s]
  0%|                                                  | 0/2039 [00:00<?, ?it/s][15:18:44] WARNING: not removing hydrogen atom without neighbors
[15:18:44] WARNING: not removing hydrogen atom without neighbors
[15:18:44] WARNING: not removing hydrogen atom without neighbors
[15:18:44] WARNING: not removing hydrogen atom without neighbors
 14%|█████▏                                | 281/2039 [00:00<00:00, 2807.75it/s][15:18:44] WARNING: not removing hydrogen atom without neighbors
[15:18:44] WARNING: not removing hydrogen atom without neighbors
[15:18:44] WARNING: not removing hydrogen atom without neighbors
[15:18:44] WARNING: not removing hydrogen atom without neighbors
[15:18:44] WARNING: not removing hydrogen atom without neighbors
[15:18:44] WARNING: not removing hydrogen atom without neighbors
 28%|██████████▍                           | 562/2039 [00:00<00:00, 2800.31it/s][15:18:44] WARNING: not removing hydrogen atom without neighbors
[15:18:44] WARN

In [7]:
data_process

In [13]:
type(data_process)

chemprop.data.data.MoleculeDataset

In [9]:
import chemprop
# Constructing MPNN
from chemprop.args import TrainArgs
arguments = [
    '--data_path', 'data/regression.csv',
    '--dataset_type', 'regression',
    '--save_dir', 'test_checkpoints_reg',
    '--epochs', '5',
    '--save_smiles_splits'
]

args = chemprop.args.TrainArgs().parse_args(arguments)
mpnn = chemprop.models.MoleculeModel(args)

/home/zhengyjo/anaconda3/lib/python3.7/site-packages/torch/nn/init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


In [11]:
PATH = '../../test_checkpoints_reg/fold_0/model_0/model.pt'

model = model = load_checkpoint(PATH, device=args.device)

Loading pretrained parameter "encoder.encoder.0.cached_zero_vector".
Loading pretrained parameter "encoder.encoder.0.W_i.weight".
Loading pretrained parameter "encoder.encoder.0.W_h.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.weight".
Loading pretrained parameter "encoder.encoder.0.W_o.bias".
Loading pretrained parameter "readout.1.weight".
Loading pretrained parameter "readout.1.bias".
Loading pretrained parameter "readout.4.weight".
Loading pretrained parameter "readout.4.bias".
Moving model to cuda


In [24]:
batch = data_process

In [26]:
mol_batch = batch.batch_graph()
features_batch = batch.features()
target_batch = batch.targets()
mask_batch = batch.mask()
atom_descriptors_batch = batch.atom_descriptors()
atom_features_batch = batch.atom_features()
bond_descriptors_batch = batch.bond_descriptors()
bond_features_batch = batch.bond_features()
constraints_batch = batch.constraints()
data_weights_batch = batch.data_weights()

In [29]:
bond_types_batch = None
bond_types_batch = None

In [43]:
preds = model.encoder(
            mol_batch
        )

In [44]:
preds.shape

torch.Size([2039, 300])

In [37]:
bond_features_batch

In [39]:
model.encoder()

torch.Size([2039, 1])

# Set up a new model for contrastive learning


In [53]:
from chemprop.models.mpn_contrastive import MPNCEncoder

ModuleNotFoundError: No module named 'chemprop.models.mpn_contrastive'

In [47]:
mpnc = chemprop.models.mpn_contrastive(args)

AttributeError: module 'chemprop.models' has no attribute 'mpn_contrastive'